In [79]:
import numpy as np
import numpy.polynomial.legendre as leg
import scipy as sc
from scipy.stats import norm
from scipy import optimize as op
import math
import cmath
import matplotlib.pyplot as plt

# Simulation of stochastic volatility models

This notebook is a companion to the lecture given as part of the 11th Summer School in Mathematical Finance on the 21st of February 2018 at the African Insitute for Mathematical Sciences. In several exercises in this notebook you will learn the pitfalls of simulating stochastic volatility models, using basic Euler schemes.

## Black-Scholes model

Before starting out with stochastic volatility models, we will introduce some basic routines that can be used for simulation. We will focus here on the Black-Scholes model, for which we obviously have a closed-form solution. The closed-form (forward, i.e. undiscounted) option price is implemented as forward_opt below.

In [3]:
class BlackScholes:
    def __init__(self, mu, vol):
        self.mu = mu
        self.vol = vol

    def forward_opt(self, forward, strike, maturity):
        d1 = (math.log(forward / strike) + 0.5 * self.vol * self.vol * maturity) / (self.vol * math.sqrt(maturity))
        d2 = d1 - self.vol * math.sqrt(maturity)
        nd1 = norm.cdf(d1)
        nd2 = norm.cdf(d2)
        return forward * nd1 - strike * nd2
        
    def _simulate_increment(self, dt, randn):
        return (self.mu - 0.5 * self.vol * self.vol) * dt + self.vol * math.sqrt(dt) * randn
    
    def simulate_stock(self, spot, maturity, num_steps):
        dt = maturity / num_steps
        increment = sum(map(lambda x: self._simulate_increment(dt, sc.randn()), range(num_steps)))
        return spot * math.exp(increment)

The next routine prices a call option, taking the model (so far we only have Black-Scholes available), strike, maturity, number of steps and the number of paths as input. It will output the sample mean and the standard deviation of the sample mean.

In [4]:
def price_call_option(model, spot, strike, maturity, num_steps, num_paths):
    realisations = list(map(lambda x: max(model.simulate_stock(spot, maturity, num_steps) - strike, 0.0),
                            range(num_paths)))
    return np.average(realisations), 1.0 / math.sqrt(num_paths) * np.std(realisations)

**Q:** Please test that this works well for the Black-Scholes model, using the closed-form solution.

**A:** we can test this using the following configuration of the Black-Scholes model:

In [5]:
bs = BlackScholes(0.0, 0.2)

Simulation gives the following price + standard deviation of the estimate, for a call, struck at 100.0, maturing in 5 years time:

In [6]:
price_call_option(model=bs, spot = 100.0, strike=100.0, maturity=5.0, num_steps=1, num_paths=100000)

(17.847331702749521, 0.10950601003033374)

whereas the closed-form solution is (using a forward equal to the spot, as we chose $\mu$ to be zero):

In [7]:
bs.forward_opt(forward=100.0, strike=100.0, maturity=5.0)

17.693672624187855

Note that we only used one timestep in the simulation, as there is no discretisation error in how we simulate the Black-Scholes model. It is good to check whether the closed-form solution is in the 95% c.i. of the option price estimate.

## Heston model

In the same structure as above we have added a very basic class for simulating the Heston model.
The equations for the Heston model are the following:

$$S(t)=\mu S(t) dt + \sqrt{V(t)} S(t) dW_S(t)$$
$$V(t)=-\kappa (V(t) - \theta) dt + \omega \sqrt{V(t)} dW_V(t)$$

In [8]:
class Heston:
    def __init__(self, mu, kappa, theta, omega, rho, initial_variance):
        self.mu = mu
        self.kappa = kappa
        self.theta = theta
        self.omega = omega
        self.rho = rho
        self.initial_variance = initial_variance
    
    def _simulate_variance_next_step(self, var_prev, dt, randn):
        tmp = self.kappa * dt
        variance = (1.0 - tmp) * var_prev + tmp * self.theta + self.omega * math.sqrt(var_prev * dt) * randn
        return variance
    
    def _simulate_log_asset_next_step(self, log_asset_prev, var_prev, dt, randn):
        increment = (self.mu - 0.5 * var_prev) * dt + math.sqrt(var_prev * dt) * randn
        return log_asset_prev + increment
    
    def simulate_stock(self, spot, maturity, num_steps):
        dt = maturity / num_steps
        var_prev = self.initial_variance
        log_asset_prev = math.log(spot)
        for i in range(num_steps):
            randn_vol = sc.randn()
            randn_spot = self.rho * randn_vol + math.sqrt(1.0 - self.rho * self.rho) * sc.randn()
            var_next = self._simulate_variance_next_step(var_prev, dt, randn_vol)
            log_asset_next = self._simulate_log_asset_next_step(log_asset_prev, var_prev, dt, randn_spot)
            log_asset_prev = log_asset_next
            var_prev = var_next
        return math.exp(log_asset_next)      

The class follows the same structure as the Black-Scholes class, it has a simulate_stock method which produces the value of the stock at maturity. Therefore we can use this class in the price_call_option function we defined earlier.

**Q:** First of all please test that the above implementation reproduces the results for the Black-Scholes model.

**A:** When setting $\kappa = 0, \omega = 0, V(0) = \sigma^2$, we have reduced the Heston model to the Black-Scholes model. Therefore we can test if the same call option, priced in this model, has the same value as we obtained before.

In [9]:
heston_special_case = Heston(mu=0.0, kappa=0.0, omega=0.0, rho=0.0, theta=0.0, initial_variance=0.2*0.2)

In [10]:
price_call_option(model=heston_special_case, spot=100.0, strike=100.0, maturity=5.0, num_steps=1, num_paths=100000)

(17.617437792772058, 0.108374817593249)

**Q:** Second of all, start working on the following example:

$$\kappa = 2, \omega = 1, \rho = -0.3, \theta = 0.09, V(0) = 0.09$$

and price a 5y call option struck at 100, where the initial spot is 100 and the drift is 5% per year. Does the above implementation work for this example? Why not?
Can you measure in what percentage of the paths there is a problem?

**A:** To test this, we need to define the Heston model. The code for this was already given below:

In [11]:
heston = Heston(mu=0.05, kappa=2.0, omega=1.0, rho=-0.3, theta=0.09, initial_variance=0.09)

Thereafter we will try to price the requested option, using the price_call_option function:

In [12]:
price_call_option(heston, spot=100.0, strike=100.0, maturity=5.0, num_steps=10, num_paths=10000)

ValueError: math domain error

We get an error - math domain error. This probably is caused by the fact that the variance becomes negative. What we will do is fix the problem, and measure how often it occurs by keeping track of the occurence of negative variances in a member variable. We will use the "absorption" fix.

In [76]:
class Heston_Absorption(Heston):
    def __init__(self, mu, kappa, theta, omega, rho, initial_variance):
        super().__init__(mu, kappa, theta, omega, rho, initial_variance)
        self.number_of_negative_variances = 0
    
    def _simulate_variance_next_step(self, var_prev, dt, randn):
        if var_prev > 0.0:
            self.number_of_negative_variances = self.number_of_negative_variances + 1
        var_prev_adjusted = max(var_prev, 0.0)
        tmp = self.kappa * dt
        variance = (1.0 - tmp) * var_prev_adjusted + tmp * self.theta + self.omega * math.sqrt(var_prev_adjusted * dt) * randn
        return max(variance, 0.0)

Let us now define this implementation of the Heston absorption scheme and initialise our counting variable:

In [77]:
heston_abs = Heston_Absorption(mu=0.05, kappa=2.0, omega=1.0, rho=-0.3, theta=0.09, initial_variance=0.09)

We can price the call option in a similar way:

In [78]:
price_call_option(model=heston_abs, spot=100.0, strike=100.0, maturity=5.0, num_steps=10, num_paths=10000)

(53.333336787183896, 1.2846382676509487)

We have stored the number of occurrences of a negative variance in a global variable - that, divided by the number of paths and the number of timesteps, tells us what percentage of time the variance has to be "fixed".

In [ ]:
heston_abs.number_of_negative_variances / (10 * 10000)

**Q:** Please adapt the implementation to get it to work - you can use one of the schemes initially considered in the slides - i.e. absorption or reflection. The benchmark price for this option is 44.94063.

**A:** We have just done this. We are quite far from the benchmark price though. Increasing the number of timesteps obviously helps:

In [ ]:
price_call_option(model=heston_abs, spot=100.0, strike=100.0, maturity=5.0, num_steps=100, num_paths=100000)

Even with 100 timesteps we are still quite far from the true price - but luckily the above result matches the reference biases produced for this example (the SV-I example) from the Lord, Koekkoek and Van Dijk paper (http://www.rogerlord.com/biasedsv.pdf). The only difference is that we don't discount the price here.

**Q:** Once you have got it to work - try to adapt your implementation to use the full truncation scheme and compare it to the scheme you used previously. How does it perform in comparison?

**A:** The full truncation implementation is given below - as you can see we do not simply set the variance to zero when it is negative, but allow it to remain negative. Only in the drift and diffusion parts to we use a truncated version.

In [ ]:
class Heston_Full_Truncation(Heston):
    def _simulate_variance_next_step(self, var_prev, dt, randn):
        var_prev_adjusted = max(var_prev, 0.0)
        tmp = self.kappa * dt
        variance = var_prev - tmp * (var_prev_adjusted - self.theta) + self.omega * math.sqrt(var_prev_adjusted * dt) * randn
        return variance
    
    def _simulate_log_asset_next_step(self, log_asset_prev, var_prev, dt, randn):
        var_prev_adjusted = max(var_prev, 0.0)
        increment = (self.mu - 0.5 * var_prev_adjusted) * dt + math.sqrt(var_prev_adjusted * dt) * randn
        return log_asset_prev + increment

In [ ]:
heston_ft = Heston_Full_Truncation(mu=0.05, kappa=2.0, omega=1.0, rho=-0.3, theta=0.09, initial_variance=0.09)
price_call_option(model=heston_ft, spot=100.0, strike=100.0, maturity=5.0, num_steps=100, num_paths=100000)

**A:** The above answer is already much more in line with the benchmark price we supplied earlier (44.94), and - depending on your particular draws of random numbers - probably already statistically indistinguishable from the benchmark price.

**Q:** Can you measure the "leaking correlation" problem in this Euler discretisation? How far is the correlation between the log-asset increment and the variance increment from what it should be?

**A:** What we will do is the following - we will keep track of the simulated variances and stock values in an internal dataframe, so that we can run this analysis after the simulation.

In [73]:
class Heston_Full_Truncation_With_Correlation_Measurement(Heston_Full_Truncation):  
    def __init__(self, mu, kappa, theta, omega, rho, initial_variance):
        super().__init__(mu, kappa, theta, omega, rho, initial_variance)

        self.num_paths = 0
        self.avg_correlation = 0.0
        self._clear_statistics()
    
    def _clear_statistics(self):
        self.log_asset_incr_first_mom = 0.0
        self.log_asset_incr_second_mom = 0.0
        self.var_incr_first_mom = 0.0
        self.var_incr_second_mom = 0.0
        self.cross_first_mom = 0.0
        
    def _update_statistics(self, asset_incr, var_incr, num_steps):
        self.log_asset_incr_first_mom = self.log_asset_incr_first_mom + 1.0 / num_steps * asset_incr
        self.log_asset_incr_second_mom = self.log_asset_incr_second_mom + 1.0 / num_steps * asset_incr * asset_incr
        self.var_incr_first_mom = self.var_incr_first_mom + 1.0 / num_steps * var_incr
        self.var_incr_second_mom = self.var_incr_second_mom + 1.0 / num_steps * var_incr * var_incr
        self.cross_first_mom = self.cross_first_mom + 1.0 / num_steps * asset_incr * var_incr
        
    def _calculate_correlation(self):
        self.num_paths = self.num_paths + 1
        asset_incr_stdev = math.sqrt(self.log_asset_incr_second_mom - self.log_asset_incr_first_mom * self.log_asset_incr_first_mom)
        var_incr_stdev = math.sqrt(self.var_incr_second_mom - self.var_incr_first_mom * self.var_incr_first_mom)
        cov = self.cross_first_mom - self.log_asset_incr_first_mom * self.var_incr_first_mom
        return cov / (asset_incr_stdev * var_incr_stdev)
    
    def simulate_stock(self, spot, maturity, num_steps):
        self._clear_statistics()
        
        dt = maturity / num_steps
        var_prev = self.initial_variance
        log_asset_prev = math.log(spot)
        for i in range(num_steps):
            randn_vol = sc.randn()
            randn_spot = self.rho * randn_vol + math.sqrt(1.0 - self.rho * self.rho) * sc.randn()
            var_next = self._simulate_variance_next_step(var_prev, dt, randn_vol)
            log_asset_next = self._simulate_log_asset_next_step(log_asset_prev, var_prev, dt, randn_spot)
            self._update_statistics(log_asset_next - log_asset_prev, var_next - var_prev, num_steps)
            log_asset_prev = log_asset_next
            var_prev = var_next
        
        # Here we calculate the correlation for the current path, between the increments of the log_asset and the variance
        corr = self._calculate_correlation()
        
        # We also keep track of the average over all simulated paths thus far.
        self.avg_correlation = (self.avg_correlation * (self.num_paths - 1) + corr) / self.num_paths
        
        return math.exp(log_asset_next)

Now we run the same code again:

In [74]:
heston_ft = Heston_Full_Truncation_With_Correlation_Measurement(mu=0.05, kappa=2.0, omega=1.0, rho=-0.3, theta=0.09, initial_variance=0.09)
price_call_option(model=heston_ft, spot=100.0, strike=100.0, maturity=5.0, num_steps=100, num_paths=100000)

(45.028786870199589, 0.23447892908633103)

The average correlation can be accessed as follows:

In [75]:
heston_ft.avg_correlation

-0.2821394775860415

As we can see, the average correlation is here quite close to -30%, as expected - there is virtually no bias left. However, had we used a much smaller amount of timesteps, we would have witnessed "leaking correlation":

In [57]:
heston_ft = Heston_Full_Truncation(mu=0.05, kappa=2.0, omega=1.0, rho=-0.3, theta=0.09, initial_variance=0.09)
price_call_option(model=heston_ft, spot=100.0, strike=100.0, maturity=5.0, num_steps=5, num_paths=100000)
heston_ft.avg_correlation

-0.05208783424429148